In [ ]:
import numpy as np

class Counter(dict):
    def increment(self, item, delta = 1):
        new_val = delta + self.pop(item, 0)
        if new_val > 0:
            self[item] = new_val

def y(x):
    return 2**x

def find_tranformation_matrix(b, E, N, Tmax, stopping_crit = 0):

    M = np.shape(E)[0]
    ## find the number of events that occur in the interval from T = 0 to T = Tmax
    n = np.random.poisson(lam = Tmax*(N+b*M))

    ## find times for all events
    times = np.sort(np.random.uniform(0, Tmax, n))
    R = np.zeros(n, dtype = int)

    ##making a spatial plot
    spatial1 = np.zeros(n, dtype = int)
    spatial2 = np.zeros(n, dtype = int)
    
    X = [2**i for i in range(N)][::-1]

    counts = Counter()
    for element in X:
        counts.increment(element, 1)

    # run simulation
    for t in range(n):
        
        if np.random.random() < N/(N + b*M):

            i = np.random.randint(N)

            if X[i] != 0:
                
                counts.increment(X[i], -1)

                X[i] = 0

                counts.increment(0, 1)
        else:

            i,j = E[np.random.randint(M)]

            if (X[i] != 0 or X[j] != 0) and X[i] != X[j]:
                    
                new = X[i] | X[j]
                counts.increment(X[i], -1)
                counts.increment(X[j], -1)
                counts.increment(new, 2)
                X[i] = X[j] = new
                
        R[t] = non_zero_distinct_rows = len(counts) - (0 in counts)
        
        ## make duplicate dicionary to edit
        sorted_counts = counts
        ## will never have 0 be the most commonly occuring element
        sorted_counts[0] = -1
        sorted_counts = sorted(counts.items(), key=lambda item: item[1], reverse=True)
        ## most commonly occuring
        spatial1[t] = sorted_counts[0][1]
        ## second most commonly occuring
        spatial2[t] = sorted_counts[1][1]
        
        ## break the simulation because the pseudo mixing time has been found (the number of distinct rows and thus distinct columns is 2 or extinction, depending)

        if non_zero_distinct_rows == stopping_crit:
            times = times[:t+1]
            R = R[:t+1]
            break

    return times, R, spatial1, spatial2


In [ ]:
N =  6400
M = 3*N
Tmax = 500
## at the transition -- try again with the epidemic 
b = 0.22

## random edge matrix
E = np.random.randint(0,N,size=2*M).reshape((N*3, 2))

## run all the way to extinction time
## average over 10 trials per value of b
## find peak of the spatial1 and spatial2 lists
## plot these maximum points against the b values
times, R, spatial1, spatial2 = find_tranformation_matrix(b, E, N, Tmax, 0)

